In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"

  # IPython 7.14 deprecated set_matplotlib_formats from IPython
  try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
  except ImportError:
    # Fall back to deprecated location for older IPython versions
    from IPython.display import set_matplotlib_formats
    
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L2hvbWUvbWlrZW5ndXllbi9wcm9qZWN0L3RpZHlmaW5hbmNl'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"/opt/anaconda3/lib/python3.13/importlib/_bootstrap.py": 1749745530.584952, "/opt/anaconda3/lib/python3.13/importlib/_bootstrap_external.py": 1749745530.5809522, "/opt/anaconda3/lib/python3.13/zipimport.py": 1749745530.2849557, "/opt/anaconda3/lib/python3.13/codecs.py": 1749745530.164957, "/opt/anaconda3/lib/python3.13/encodings/aliases.py": 1749745530.3809545, "/opt/anaconda3/lib/python3.13/encodings/__init__.py": 1749745530.428954, "/opt/anaconda3/lib/python3.13/encodings/utf_8.py": 1749745530.4809532, "/opt/anaconda3/lib/python3.13/abc.py": 1749745530.1569571, "/opt/anaconda3/lib/python3.13/io.py": 1749745530.2049568, "/opt/anaconda3/lib/python3.13/stat.py": 1749745530.256956, "/opt/anaconda3/lib/python3.13/_collections_abc.py": 1749745530.168957, "/opt/anaconda3/lib/python3.13/genericpath.py": 1749745530.1929567, "/opt/anaconda3/lib/python3.13/posixpath.py": 1749745530.2289565, "/opt/anaconda3/lib/python3.13/os.py": 1749745530.2209566, "/opt/anaconda3/lib/python3.13/_sitebuiltins.

In [2]:
import pandas as pd
import numpy as np

In [3]:
import requests
# print(requests.get("https://api.ipify.org").text)

In [4]:
import socket

def get_private_ip():
    try:
        # Create a UDP socket
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        
        # Connect to a public DNS server (Google's 8.8.8.8)
        # We use port 80, but the specific port doesn't matter here
        s.connect(("8.8.8.8", 80))
        
        # Get the socket's own address
        private_ip = s.getsockname()[0]
        
        # Close the socket
        s.close()
        
        return private_ip
    except Exception as e:
        return f"Error: {e}"

# if __name__ == "__main__":
#     print(f"Your Private IP is: {get_private_ip()}")

In [5]:
import requests
import json
import os
import pandas as pd
import time

# Configuration
URL = "https://gateway.datacore.vn/data/ds/search"
API_KEY = os.environ.get('datacore_api') # Ensure this environment variable is set

if not API_KEY:
    raise ValueError("API Key not found. Please set the 'datacore_api' environment variable.")

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

def get_full_dataset():
    all_rows = []
    columns = []
    
    current_page = 1
    total_pages = 1 # Will be updated after the first request
    limit = 100     # Increase limit to reduce number of requests (adjust if API rejects high limits)

    print("Starting data retrieval...")

    while current_page <= total_pages:
        payload = {
            "dataSetCode": "fundamental_annual", # Correct dataset code
            "conditions": [],                    # Empty list to get ALL data
            "selectFields": [],                  # Empty list to get ALL columns
            "page": current_page,
            "limit": limit
        }

        try:
            response = requests.post(URL, headers=headers, json=payload)
            response.raise_for_status() # Raises error for 4xx or 5xx status codes
            data = response.json()

            # On the first page, capture total pages and column names
            if current_page == 1:
                total_pages = data.get("totalPage", 0)
                columns = data.get("fields", [])
                print(f"Total Pages to fetch: {total_pages}")
                print(f"Columns found: {columns}")

            # Extract row data
            page_data = data.get("dataDetail", [])
            
            if not page_data:
                print(f"No data returned on page {current_page}. Stopping.")
                break

            all_rows.extend(page_data)
            print(f"Fetched page {current_page}/{total_pages} - Total rows so far: {len(all_rows)}")
            
            current_page += 1
            
            # Optional: Sleep briefly to avoid hitting rate limits
            time.sleep(0.1)

        except requests.exceptions.RequestException as e:
            print(f"Request failed on page {current_page}: {e}")
            break
        except json.JSONDecodeError:
            print(f"Failed to decode JSON on page {current_page}")
            print("Response text:", response.text)
            break

    # Create DataFrame
    if all_rows and columns:
        df = pd.DataFrame(all_rows, columns=columns)
        return df
    else:
        print("No data retrieved.")
        return None

# Execution
if __name__ == "__main__":
    df = get_full_dataset()
    
    if df is not None:
        print("\n===== FINAL DATASET =====")
        print(df.info())
        print(df.head())

Starting data retrieval...


Total Pages to fetch: 0
Columns found: []
No data returned on page 1. Stopping.
No data retrieved.


In [6]:
import requests
import json

url = "https://gateway.datacore.vn/data/ds/search"

import os
datacore_api = os.environ['datacore_api']

headers = {
    "x-api-key": datacore_api,
    "Content-Type": "application/json"
}

payload = {
    "dataSetCode": "vsic",
    "conditions": [
        {
            "field": "Code",
            "operator": "EQUAL",
            "conditionValue": "52244"
        }
    ],
    "selectFields": [
        "Name",
        "Level"
    ],
    "page": 1,
    "limit": 10
}

# ===== LOG REQUEST =====
print("===== REQUEST =====")
print("URL:", url)
print("Headers:", json.dumps(headers, indent=2, ensure_ascii=False))
print("Body:", json.dumps(payload, indent=2, ensure_ascii=False))

# Gửi request
response = requests.post(url, headers=headers, json=payload)

# ===== LOG RESPONSE =====
print("\n===== RESPONSE =====")
print("Status Code:", response.status_code)

try:
    print("Response JSON:")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))
except Exception:
    print("Response Text:")
    print(response.text)

===== REQUEST =====
URL: https://gateway.datacore.vn/data/ds/search
Headers: {
  "x-api-key": "YzVmYTVlNzQtOTI1NS00ZDQ5LWEwNmQtYjMzNzZiZTUxMjVl",
  "Content-Type": "application/json"
}
Body: {
  "dataSetCode": "vsic",
  "conditions": [
    {
      "field": "Code",
      "operator": "EQUAL",
      "conditionValue": "52244"
    }
  ],
  "selectFields": [
    "Name",
    "Level"
  ],
  "page": 1,
  "limit": 10
}

===== RESPONSE =====
Status Code: 200
Response JSON:
{
  "status": true,
  "message": "success",
  "httpCode": 200,
  "errorCode": "ok",
  "data": {
    "fields": [
      "Name",
      "Level"
    ],
    "dataDetail": [
      [
        "Bốc xếp hàng hóa cảng sông",
        5
      ]
    ],
    "num": 1,
    "totalPage": 1,
    "currentPage": 1
  }
}


In [7]:
import os
import boto3
from botocore.client import Config


class ObjectStorage:
    def __init__(self):
        self.client = boto3.client(
            "s3",
            endpoint_url=os.environ["MINIO_ENDPOINT"],
            aws_access_key_id=os.environ["MINIO_ACCESS_KEY"],
            aws_secret_access_key=os.environ["MINIO_SECRET_KEY"],
            region_name=os.getenv("MINIO_REGION", "us-east-1"),
            config=Config(signature_version="s3v4"),
        )


storage = ObjectStorage()
bucket = os.environ["MINIO_BUCKET"]

In [8]:
from io import BytesIO

prices = pd.read_csv(
    BytesIO(
        storage.client.get_object(
            Bucket=bucket,
            Key="historycal_price/dataset_historical_price.csv",
        )["Body"].read()
    ),
    low_memory=False,
)


prices["date"] = pd.to_datetime(prices["date"])


prices["adjusted_close"] = prices["close_price"] * prices["adj_ratio"]


prices = prices.rename(
    columns={
        "vol_total": "volume",
        "open_price": "open",
        "low_price": "low",
        "high_price": "high",
        "close_price": "close",
    }
)

prices = prices.sort_values(["symbol", "date"])

In [9]:
import datetime as dt

start = pd.Timestamp("2000-01-01")
end = pd.Timestamp(dt.date.today().year - 1, 12, 31)


fpt = prices.loc[
    (prices["symbol"] == "FPT")
    & (prices["date"] >= start)
    & (prices["date"] <= end),
    ["date", "symbol", "volume", "open", "low", "high", "close", "adjusted_close"],
].copy()

In [10]:
from plotnine import ggplot, aes, geom_line, labs

In [11]:
#| label: fig-100
#| fig-cap: Prices are in VND, adjusted for dividend payments and stock splits.
#| fig-alt: 'Title: FPT stock prices'
(
    ggplot(fpt, aes(x="date", y="adjusted_close"))
    + geom_line()
    + labs(title="Adjusted price path of FPT", x="", y="")
)

In [12]:
prices.loc[prices["adjusted_close"] <= 0, ["symbol", "date", "adjusted_close"]].head()

,symbol,date,adjusted_close
33886,ADP,2010-02-09,0.0
33887,ADP,2010-02-24,0.0
33888,ADP,2010-03-01,0.0
33889,ADP,2010-03-03,0.0
33890,ADP,2010-03-12,0.0


In [13]:
returns = (
    prices
    .loc[prices["adjusted_close"] > 0]
    .sort_values(["symbol", "date"])
    .assign(ret=lambda x: x.groupby("symbol")["adjusted_close"].pct_change())
    [["symbol", "date", "ret"]]
)
returns = returns.dropna(subset=["ret"])

In [14]:
def winsorize_cs(df, column="ret", lower_q=0.01, upper_q=0.99):
    lo = df[column].quantile(lower_q)
    hi = df[column].quantile(upper_q)
    out = df.copy()
    out[column] = out[column].clip(lo, hi)
    return out

returns = winsorize_cs(returns)

In [15]:
from mizani.formatters import percent_format
from plotnine import geom_histogram, geom_vline, scale_x_continuous


fpt_ret = returns.loc[returns["symbol"] == "FPT"].copy()
q05 = fpt_ret["ret"].quantile(0.05)

In [16]:
#| label: fig-101
#| fig-alt: 'Title: Distribution of daily FPT stock returns in percent. The figure shows a histogram of daily returns. The vertical line indicates that the historical five percent quantile of daily returns was around negative three percent.'
#| fig-cap: The dotted vertical line indicates the historical five percent quantile.
(
    ggplot(fpt_ret, aes(x="ret"))
    + geom_histogram(bins=100)
    + geom_vline(xintercept=q05, linetype="dashed")
    + scale_x_continuous(labels=percent_format())
    + labs(title="Distribution of daily FPT returns", x="", y="")
)

In [17]:
returns["ret"].describe().round(3)

count    4305063.000
mean           0.000
std            0.035
min           -0.125
25%           -0.004
50%            0.000
75%            0.003
max            0.130
Name: ret, dtype: float64

In [18]:
(
    returns
    .assign(year=lambda x: x["date"].dt.year)
    .groupby("year")["ret"]
    .describe()
    .round(3)
)

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2010,131548.0,-0.001,0.036,-0.125,-0.021,0.0,0.018,0.13
2011,166826.0,-0.003,0.033,-0.125,-0.020,0.0,0.011,0.13
2012,177938.0,0.000,0.033,-0.125,-0.012,0.0,0.015,0.13
2013,180417.0,0.001,0.033,-0.125,-0.004,0.0,0.008,0.13
2014,181907.0,0.001,0.034,-0.125,-0.008,0.0,0.011,0.13
2015,197881.0,0.000,0.033,-0.125,-0.006,0.0,0.005,0.13
2016,227896.0,0.000,0.035,-0.125,-0.005,0.0,0.003,0.13
2017,283642.0,0.001,0.034,-0.125,-0.002,0.0,0.001,0.13
2018,329887.0,0.000,0.035,-0.125,0.000,0.0,0.000,0.13


In [19]:
vn30 = [
    "ACB","BCM","BID","BVH","CTG","FPT","GAS","GVR","HDB","HPG",
    "MBB","MSN","MWG","PLX","POW","SAB","SHB","SSB","STB","TCB",
    "TPB","VCB","VHM","VIB","VIC","VJC","VNM","VPB","VRE","EIB",
]


prices_vn30 = prices.loc[prices["symbol"].isin(vn30)]
from plotnine import theme

In [20]:
#| label: fig-102
#| fig-cap: Prices in VND, adjusted for dividend payments and stock splits.
#| fig-alt: 'Title: Stock prices of VN30 index constituents.'
(
    ggplot(prices_vn30, aes(x="date", y="adjusted_close", color="symbol"))
    + geom_line()
    + labs(title="Adjusted prices of VN30 constituents", x="", y="")
    + theme(legend_position="none")
)

In [21]:
returns_vn30 = (
    prices_vn30
    .sort_values(["symbol", "date"])
    .assign(ret=lambda x: x.groupby("symbol")["adjusted_close"].pct_change())
    [["symbol", "date", "ret"]]
    .dropna()
)


returns_vn30.groupby("symbol")["ret"].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
symbol,,,,,,,,
ACB,3822.0,-0.000,0.023,-0.407,-0.006,0.0,0.007,0.097
BCM,1795.0,0.001,0.027,-0.136,-0.010,0.0,0.010,0.159
BID,2811.0,0.000,0.024,-0.369,-0.010,0.0,0.011,0.070
BVH,3825.0,0.000,0.024,-0.097,-0.012,0.0,0.012,0.070
CTG,3825.0,0.000,0.024,-0.376,-0.010,0.0,0.010,0.070
EIB,3825.0,-0.000,0.022,-0.302,-0.008,0.0,0.008,0.070
FPT,3825.0,-0.000,0.024,-0.439,-0.008,0.0,0.009,0.070
GAS,3236.0,0.000,0.022,-0.289,-0.009,0.0,0.010,0.070
GVR,1775.0,0.001,0.030,-0.137,-0.014,0.0,0.016,0.169


In [22]:
returns_monthly = (
    returns_vn30
    .assign(month=lambda x: x["date"].dt.to_period("M").dt.to_timestamp())
    .groupby(["symbol", "month"], as_index=False)
    .agg(ret=("ret", lambda x: np.prod(1 + x) - 1))
)

In [23]:
from plotnine import facet_wrap

fpt_d = returns_vn30.loc[returns_vn30["symbol"] == "FPT"].assign(freq="Daily")
fpt_m = returns_monthly.loc[returns_monthly["symbol"] == "FPT"].assign(freq="Monthly")


fpt_both = pd.concat([
    fpt_d[["ret", "freq"]],
    fpt_m[["ret", "freq"]],
])

In [24]:
#| label: fig-103
#| fig-cap: Returns are based on prices adjusted for dividend payments and stock splits.
#| fig-alt: 'Title: Distribution of FPT returns across different frequencies. The figure shows the distribution of daily and monthly returns in two separate facets.'
(
    ggplot(fpt_both, aes(x="ret"))
    + geom_histogram(bins=50)
    + scale_x_continuous(labels=percent_format())
    + labs(title="FPT returns at different frequencies", x="", y="")
    + facet_wrap("freq", scales="free")
)

In [25]:
trading_value = (
    prices_vn30
    .assign(value=lambda x: x["volume"] * x["adjusted_close"] / 1e9)
    .groupby("date")["value"]
    .sum()
    .reset_index()
    .assign(value_lag=lambda x: x["value"].shift(1))
)
(
    ggplot(trading_value, aes(x="date", y="value"))
    + geom_line()
    + labs(title="Aggregate VN30 trading value (billion VND)", x="", y="")
)

In [26]:
from plotnine import geom_point, geom_abline

In [27]:
#| label: fig-104
#| fig-cap: Total daily trading volume.
#| fig-alt: 'Title: Aggregate daily trading volume of VN30 index constitutens. The figure shows a volatile time series of daily trading volume.'
(
    ggplot(trading_value, aes(x="value_lag", y="value"))
    + geom_point()
    + geom_abline(intercept=0, slope=1, linetype="dashed")
    + labs(
        title="Persistence in VN30 trading value",
        x="Previous day",
        y="Current day",
    )
)

/home/mikenguyen/project/tidyfinance/.venv/lib/python3.13/site-packages/plotnine/layer.py:374: PlotnineWarning: geom_point : Removed 1 rows containing missing values.
